In [ ]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.7 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import *

In [ ]:
# Model gurobi
model=gp.Model("ConcreteDelivery")

Restricted license - for non-production use only - expires 2024-10-28


In [ ]:
# SETS
# Set of Vehicles
Vehicles = ["K1", "K2", "K3", "K4"]
VehicleCapacity = [5, 5, 5, 5]

In [ ]:
# PARAMETERS
# //Demand
Demand = [20, 5, 10]
CustomerNode = [4, 5, 6]
MaxNumDeliveryC = [4, 1, 2]
# Min time lag customer
mintlC = [10/60, 0/60, 15/60]
# max time lag customer
maxtlC = [20/60, 20/60, 15/60]
DepotNode = [0, 2]
nDepotNode = len(DepotNode)
# min time lag depot
mintlD = [30/60, 20/60]
MaxNumDeliveryD = [7, 10]

In [ ]:
#Nodes
NodeSet = [0, 1, 2, 3, 4, 5, 6]
nNodeSet = len(NodeSet)

# // NodeSetC or customer nodes or C^G = {C1_1, C1_2, C1_3, C1_4, C2_1, C3_1, C3_2}
# //                               C^G = {   0,    1,    2,    3,    4,    5,    6}

# // NodeSetD or depot nodes or D^G = {D1_1, D1_2, D1_3, D1_4, D1_5, D1_6, D1_7, D2_1, D2_2, D2_3, D2_4, D2_5, D2_6, D2_7, D2_8, D2_9, D2_10}
# //                            D^G = {   7,    8,    9,   10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,    23}

# // NodeSetO = {O1, O2, O3, O4}
# //          = {24, 25, 26, 27}
# // NodeSetF = {F1, F2, F3, F4}
# //          = {28, 29, 30, 31}
# // AllNodeSet or node set V = C^G U D^G U O U F
# // AllNodeSet or node set V =            {C1_1, C1_2, C1_3, C1_4, C2_1, C3_1, C3_2, D1_1, D1_2, D1_3, D1_4, D1_5, D1_6, D1_7, D2_1, D2_2, D2_3, D2_4, D2_5, D2_6, D2_7, D2_8, D2_9, D2_10, O1, O2, O3, O4, F1, F2, F3, F4}
# // AllNodeSetIndex or node set V =       {   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,    23, 24, 25, 26, 27, 28, 29, 30, 31}


# // int NodeSetTo[] = new int[]         {0, 1, 2, 3, 4, 5, 6};

AllNodeSetIndex = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
AllNodeSet = [4, 4, 4, 4, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 0, 0, 3, 2, 0, 0]
AllNodeSetSequence = [0, 1, 2, 3, 0, 0, 1, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -2, -2, -2, -2, -1, -1, -1, -1]
AllNodeServiceTime = [5/60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60,
                      5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60, 5/ 60]
nAllNodeSetIndex = len(AllNodeSetIndex)
NodeSetD = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

# // nah ini di constraint (10 dan 11) sepertinya digabung time windowsnya, jadi bagaimana ya pak

# //time windows [a,b]
timeStartD = [7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25]
timeEndD = [10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25, 10.25]

NodeSetC = [0, 1, 2, 3, 4, 5, 6]
timeStartC = [8.0, 8.0, 8.0, 8.0, 8.5, 8.5, 8.5]

# //first delivery deadline ==> diasumsikan 30 menit setelah time windows start
FirstDelivery = [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
timeEndC = [9.0, 9.0, 9.0, 9.0, 10.0, 10.0, 10.0]
NodeSetCD = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
NodeSetO = [24, 25, 26, 27]
NodeSetF = [28, 29, 30, 31]

nNodeSetC = len(NodeSetC)
nNodeSetD = len(NodeSetD)
nNodeSetCD = len(NodeSetCD)

# //Edges and Nodes and Costs
# // Edges Set = {O_F, O_D, D_C, C_D, D_F} while each of this edge is assigned to each vehicle. so that
# // Edges Set K or AllEdgeSet = {O_F_K1, O_F_K2, O_F_K3, O_F_K4, O_D1_1_K1, O_D1_1_K2, O_D1_1_K3, O_D1_1_K4, O_D2_1_K1, O_D2_2_K1, O_D3_3_K1, O_D_4, D_C_1, D_C_2, D_C_3, D_C_4, C_D_1, C_D_2, C_D_3, C_D_4, D_F_1, D_F_2, D_F_3, D_F_4}
# //                AllEdgeSet = {    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,    10,    11,    12,    13,    14,    15,    16,    17,    18,    19}

AllEdgeSet = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
              36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
              70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102,
              103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
              130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
              157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
              184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210,
              211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
              238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264,
              265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291,
              292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318,
              319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337]
NodesFrom = [24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
             25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
             27, 27, 27, 27, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11,
             12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16,
             16, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21,
             21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
             1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
             6, 6, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6]

NodesTo = [28, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 29, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
           30, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 31, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
           0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
           4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
           1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
           22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
           7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9,
           10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 28, 29, 30, 31, 28,
           29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31, 28, 29, 30, 31]

CostEdge = [0, 12, 12, 12, 12, 12, 12, 12, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0,
            10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 30, 10, 10, 20, 20,
            20, 20, 30, 10, 10, 20, 20, 20, 20, 30, 10, 10, 20, 20, 20, 20, 30, 10, 10, 20, 20, 20, 20, 30, 10, 10, 20, 20, 20, 20, 30, 10, 10, 20, 20,
            20, 20, 30, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10,
            10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10, 10, 10, 20, 10, 10, 10, 10,
            10, 10, 20, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10,
            10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10,
            10, 10, 10, 30, 30, 30, 30, 30, 30, 30, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
            10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12, 10, 20, 20, 12, 10, 20, 20, 12, 10, 20, 20, 12, 10, 20, 20,
            20, 20, 30, 30, 11, 10, 10, 10, 11, 10, 10, 10]

TravelTimeEdge = [0.0 / 60, 12.0 / 60, 12.0 / 60, 12.0 / 60, 12.0 / 60, 12.0 / 60, 12.0 / 60, 12.0 / 60, 2.0 / 60, 2.0 / 60, 2.0 / 60, 2.0 / 60, 2.0 / 60,
                  2.0 / 60, 2.0 / 60, 2.0 / 60, 2.0 / 60, 2.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60,
                  5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 5.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60,
                  0.0 / 60, 0.0 / 60, 0.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 0.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60,
                  20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60, 10.0 / 60, 10.0 / 60,
                  20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60, 10.0 / 60,
                  10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 30.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60,
                  20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60,
                  20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 30.0 / 60, 30.0 / 60, 30.0 / 60,
                  30.0 / 60, 30.0 / 60, 30.0 / 60, 30.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60,
                  20.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60,
                  10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 12.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60,
                  12.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 12.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 12.0 / 60, 10.0 / 60, 20.0 / 60, 20.0 / 60, 20.0 / 60,
                  20.0 / 60, 30.0 / 60, 30.0 / 60, 11.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60, 11.0 / 60, 10.0 / 60, 10.0 / 60, 10.0 / 60]

nAllEdgeSet = len(AllEdgeSet)
nNodesFrom = len(NodesFrom)
nNodesTo = len(NodesTo)

# //Edge set O_D dan O_F
# // OFEgdes =        {O_F_1, O_F_2, O_F_3, O_F_4}
# //         =        {    0,     1,     2,     3}
# // OFEdgesFrom =    {   24,    25,    26,    27}
# // OFEdgesTo =      {   28,    29,    30,    31}
# //masi ragu apakah di edge nya di replikasi untuk 4 truk apa tidak perlu
# // ODEdges =        {O_D1_1_K1, O_D1_1_K2, O_D1_1_K3, O_D1_1_K4, O_D1_2_K1, O_D1_2_K2, O_D1_2_K3, O_D1_2_K4, O_D1_3_K1, O_D1_3_K2, O_D1_3_K3, O_D1_3_K4, O_D1_4_K1, O_D1_4_K2, O_D1_4_K3, O_D1_4_K4, O_D1_5_K1, O_D1_5_K2, O_D1_5_K3, O_D1_5_K4, O_D1_6_K1, O_D1_6_K2, D1_6_K3, O_D1_6_K4, O_D1_7_K1, O_D1_7_K2, O_D1_7_K3, O_D1_7_K4, O_D2_1_K1, O_D2_1_K2, O_D2_1_K3, O_D2_1_K4, O_D2_2_K1, O_D2_2_K2, O_D2_2_K3, O_D2_2_K4, O_D2_3_K1, O_D2_3_K2, O_D2_3_K3, O_D2_3_K4, O_D2_4_K1, O_D2_4_K2, O_D2_4_K3, O_D2_4_K4, O_D2_5_K1, O_D2_5_K2, O_D2_5_K3, O_D2_5_K4,  O_D2_6_K1, O_D2_6_K2, O_D2_6_K3, O_D2_6_K4, O_D2_7_K1, O_D2_7_K2, O_D2_7_K3, O_D2_7_K4, O_D2_8_K1, O_D2_8_K2, O_D2_8_K3, O_D2_8_K4, O_D2_9_K1, O_D2_9_K2, O_D2_9_K3, O_D2_9_K4, O_D2_10_K1, O_D2_10_K2, O_D2_10_K3, O_D2_10_K4}
# // ODEdges =        {        4,         5,	        6,	   7,	      8,	 9,	   10,	      11,	 12,	    13,	       14,	  15,	     16,	17,	   18,	      19,	 20,	    21,	       22,	  23,	     24,	25,	 26,	    27,	       28,	  29,	     30,	31,	   32,	      33,	 34,	    35,	       36,	  37,	     38,	39,	   40,	      41,	 42,	    43,	       44,	  45,	     46,	47,	   48,	      49,	 50,	    51,	        52,	   53,	      54,	 55,	    56,	       57,	  58,	     59,	60,	   61,	      62,	 63,	    64,	       65,	  66,	     67,	 68,         69,         70,         71}
# // ODEdgesFrom =    {       24,         25,         26,       27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,      26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,         24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,         25,         26,       27,        24,         25,         26,         27}
# // ODEdgesTo   =    {        7,          7,          7,        7,         8,         8,         8,         8,         9,         9,         9,         9,        10,        10,        10,        10,        11,        11,        11,        11,        12,        12,      12,        12,        13,        13,        13,        13,        14,        14,        14,        14,        15,        15,        15,        15,        16,        16,        16,        16,        17,        17,        17,        17,        18,        18,        18,        18,         19,        19,        19,        19,        20,        20,        20,        20,        21,        21,        21,        21,        22,         22,         22,       22,        23,         23,         23,         23}
# // so OEdgeSet will be OFEdges U ODEdges
# // OEdgeSet =     {0,     1,     2,     3,      4,      5,	    6,	   7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21, 22,	  23,	     24,	25,	 26,	    27,	       28,	  29,	     30,	31,	   32,	      33,	 34,	    35,	       36,	  37,	     38,	39,	   40,	      41,	 42,	    43,	       44,	  45,	     46,	47,	   48,	      49,	 50,	    51,	        52,	   53,	      54,	 55,	    56,	       57,	  58,	     59,	60,	   61,	      62,	 63,	    64,	       65,	  66,	     67,	 68,         69,         70,         71}
# // OEdgeSetFrom = {24,         25,         26,       27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,      26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,         24,        25,        26,        27,        24,        25,        26,        27,        24,        25,        26,        27,        24,         25,         26,       27,        24,         25,         26,         27}
# // OEdgesSetTo   =  {        7,          7,          7,        7,         8,         8,         8,         8,         9,         9,         9,         9,        10,        10,        10,        10,        11,        11,        11,        11,        12,        12,      12,        12,        13,        13,        13,        13,        14,        14,        14,        14,        15,        15,        15,        15,        16,        16,        16,        16,        17,        17,        17,        17,        18,        18,        18,        18,         19,        19,        19,        19,        20,        20,        20,        20,        21,        21,        21,        21,        22,         22,         22,       22,        23,         23,         23,         23}
OEdgeSet = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
            36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
            70, 71]
# //int OEdgeSetNodeFrom[] = new int[] {   24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	24,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	25,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	26,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27,	27};
# //int OEdgeSetNodeTo[] = new int[]   {   28,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	29,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	30,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	31,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23};
# //Edge set menuju C_F dan O_F
# // CFEdges =    {C1_1_F1, C1_1_F2, C1_1_F3, C1_1_F4,  C1_2_F1, C1_2_F2, C1_2_F3, C1_2_F4, C1_3_F1, C1_3_F2, C1_3_F3, C1_3_F4,  C1_4_F1, C1_4_F2, C1_4_F3, C1_4_F4, C2_1_F1, C2_1_F2, C2_1_F3, C2_1_F4, C3_1_F1, C3_1_F2, C3_1_F3, C3_1_F4, C3_2_F1, C3_2_F2, C3_2_F3, C3_2_F4}
# // CFEdgeFrom = {      0,   0,    0,     0,    1,    1,    1,    1,    2,     2,     2,     2,    3,    3,    3,     3,    4,    4,    4,    4,     5,     5,     5,     5,      6,    6,    6,     6}
# // CFEdgeTo   = {     28,  29,   30,    31,   28,   29,   30,   31,   28,    29,    30,    31,   28,   29,   30,    31,   28,   29,   30,   31,    28,    29,    30,    31,     28,   29,   30,    31}
# // FEdgeSet =   {     72,  73,   74,    75,   76,   77,   78,   79,   80,    81,    82,    83,   84,   85,   86,    87,   88,   89,   90,   91,    92,    93,    94,    95,     96,   97,   98,    99}

FEdgeSet = [0, 18, 36, 54, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333,
            334, 335, 336, 337]

# //int FEdgeSetNodeFrom[] = new int[] {24,	25,	26,	27,	0,	0,	0,	0,	1,	1,	1,	1,	2,	2,	2,	2,	3,	3,	3,	3,	4,	4,	4,	4,	5,	5,	5,	5,	6,	6,	6,	6};
# //int FEdgeSetNodeTo[] = new int[]   {28,	29,	30,	31,	28,	29,	30,	31,	28,	29,	30,	31,	28,	29,	30,	31,	28,	29,	30,	31,	28,	29,	30,	31,	28,	29,	30,	31,	28,	29,	30,	31};
# //Edge set D_C
# // DCEdgesFrom = {D1_1, D1_2, D1_3, D1_4, D1_5, D1_6, D1_7, D2_1, D2_2, D2_3, D2_4, D2_5, D2_6, D2_7, D2_8, D2_9, D2_10}
# // DCEdgesFrom = {      7,      7,      7,      7,      7,      7,      7,      8,      8,      8,      8,      8,      8,      8,      9,      9,      9,      9,      9,      9,      9,      10,     10,     10,     10,     10,     10,     10,     11,     11,     11,     11,     11,     11,     11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,    23

DCEdgeSet = [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104,
             105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
             133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
             161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188,
             189, 190]

# //int DCEdgeSetNodeFrom[] = new int[]{7,	7,	7,	7,	7,	7,	7,	8,	8,	8,	8,	8,	8,	8,	9,	9,	9,	9,	9,	9,	9,	10,	10,	10,	10,	10,	10,	10,	11,	11,	11,	11,	11,	11,	11,	12,	12,	12,	12,	12,	12,	12,	13,	13,	13,	13,	13,	13,	13,	14,	14,	14,	14,	14,	14,	14,	15,	15,	15,	15,	15,	15,	15,	16,	16,	16,	16,	16,	16,	16,	17,	17,	17,	17,	17,	17,	17,	18,	18,	18,	18,	18,	18,	18,	19,	19,	19,	19,	19,	19,	19,	20,	20,	20,	20,	20,	20,	20,	21,	21,	21,	21,	21,	21,	21,	22,	22,	22,	22,	22,	22,	22,	23,	23,	23,	23,	23,	23,	23};
# //int DCEdgeSetNodeTo[] = new int[]  {0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6,	0,	1,	2,	3,	4,	5,	6};

CDEdgeSet = [191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
             219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
             247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274,
             275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302,
             303, 304, 305, 306, 307, 308, 309]

# //int CDEdgeSetNodeFrom[] = new int[]{ 0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	2,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	3,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	4,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	5,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6,	6};
# //int CDEdgeSetNodeTo[]   = new int[]{ 7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23,	7,	8,	9,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19,	20,	21,	22,	23};

# //Travel Time (considered berbanding lurus dengan distance)
# //int TravelTime[] = new int[]{0, 5, 10, 12, 20, 30, 10, 5, 0, 5, 7, 25, 25, 15, 10, 5, 0, 2, 10, 20, 10, 12, 7, 2, 0, 12, 20, 11, 20, 25, 10, 12, 0, 30, 20, 30, 25, 20, 20, 30, 0, 30, 10, 15, 10, 11, 20, 30, 0};
# //Travel Cost
# //int TravelCost[] = new int[]       {1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1,	1};
# //max number of delivery(s) needed to fulfill demand = Demand (each) devided by max truck capacity
#//masih bingung in buat apa

nCustomerNode = len(CustomerNode)
nVehicles = len(Vehicles)

# //int nNodeSetFrom = NodeSetFrom.length;
# //int nNodeSetTo = NodeSetTo.length;

nNodeSetO = len(NodeSetO)
nNodeSetF = len(NodeSetF)

# //int nNodeSetDC = NodeSetDC.length;
nOEdgeSet = len(OEdgeSet)
nFEdgeSet = len(FEdgeSet)

# //int nAllNodeSetTo = AllNodeSetTo.length;
# //int nAllNodeSetFrom = AllNodeSetFrom.length;
nDemand = len(Demand)

# //maximum time concrete
MaxTimeConcrete = 90.0 / 60

# //penalty cost
# //edited by komar (diperbesar)
penalty = [1000, 1000, 1000]

In [ ]:
#//Variable
#//Decision variable, x[u][v][k] == 1 jika truk k loading di u, unloading di u, atau berangkat dari u menuju v

GRBVar = model.addVars(nNodesFrom, nNodesTo, nVehicles, name="GRBVar", vtype=GRB.INTEGER)
x = {}
for i in range(nAllEdgeSet):
    u = NodesFrom[i]
    v = NodesTo[i]
    x[u] = {}
    for k in range(nVehicles):
        x[u][v] = {}
        x[u][v][k] = model.addVar(0, 1, CostEdge[i], GRB.BINARY, "x" + "[" + str(u) + "][" + str(v) + "][" + str(k) + "]")

            # Decision time variable, w[u] == time jika x == 1, == undefined jika x == 0
# Asumsi u adalah nodeSetFrom yaitu all node selain F
w = {}
for u in range(nAllNodeSetIndex):
    w[u] = model.addVar(0, GRB.INFINITY, 0, GRB.CONTINUOUS, "w" + "[" + str(u) + "]")

# Decision variable, y[q] == 1 jika demand customer c terpenuhi, == 0 sebaliknya
# edited by komar (kurang negative)
y = {}
for q in range(nDemand):
    y[q] = model.addVar(0, 1, -penalty[q], GRB.BINARY, "y" + "[" + str(q) + "]")


In [ ]:
# Objective function to minimize total traveling cost and penalty cost
model.setObjective(model.getObjective(), GRB.MINIMIZE)

# Update model to integrate new variables
model.update()

In [ ]:
# Constraint 2 : Memastikan truk meninggalkan node O satu kali
for k in range(nVehicles):
    cons2 = gp.LinExpr()
    u = NodeSetO[k]
    for i in range(nOEdgeSet):
        if NodesFrom[OEdgeSet[i]] == u:
            v = NodesTo[OEdgeSet[i]]
            cons2.add(x[u][v][k])
    model.addConstr(cons2, gp.EQUAL, 1, "cons2"+str(u)+str(k))
for k in range(nVehicles):
    for l in range(nVehicles):
        if k != l:
            cons2B = gp.LinExpr()
            for m in range(nAllEdgeSet):
                if NodesFrom[m] == NodeSetO[l]:
                    v = NodesTo[m]
                    cons2B.add(x[NodesFrom[m],v,k])
            model.addConstr(cons2B, gp.EQUAL, 0, "cons2B"+str(k)+"_"+str(l))

KeyError: ignored

In [ ]:
 # Constraint 1 : Setiap node harus dikunjungi oleh tepat satu truk
for i in range(nNodes):
    cons1 = gb.LinExpr()
    for k in range(nVehicles):
        for j in range(nNodes):
            if i != j:
                cons1.add(x[i,j,k])
        model.addConstr(cons1, gb.EQUAL, 1, "cons1"+str(i))

NameError: ignored

In [ ]:



# Constraint 2 : Memastikan truk meninggalkan node O satu kali
for k in range(nVehicles):
    cons2 = gb.LinExpr()
    u = NodeSetO[k]
    for i in range(nOEdgeSet):
        if NodesFrom[OEdgeSet[i]] == u:
            v = NodesTo[OEdgeSet[i]]
            cons2.add(x[u,v,k])
    model.addConstr(cons2, gb.EQUAL, 1, "cons2"+str(u)+str(k))
for k in range(nVehicles):
    for l in range(nVehicles):
        if k != l:
            cons2B = gb.LinExpr()
            for m in range(nAllEdgeSet):
                if NodesFrom[m] == NodeSetO[l]:
                    v = NodesTo[m]
                    cons2B.add(x[NodesFrom[m],v,k])
            model.addConstr(cons2B, gb.EQUAL, 0, "cons2B"+str(k)+"_"+str(l))

# Constraint 3 : Memastikan truk memasuki node F satu kali
for k in range(nVehicles):
    cons3 = gb.LinExpr()
    v = NodeSetF[k]
    for i in range(nFEdgeSet):
        if NodesTo[FEdgeSet[i]] == v:
            u = NodesFrom[FEdgeSet[i]]
            cons3.add(x[u,v,k])
    model.addConstr(cons3, gb.EQUAL, 1, "cons3"+str(v)+str(k))
for k in range(nVehicles):
    for l in range(nVehicles):
        if k != l:
            cons3B = gb.LinExpr()
            for m in range(nAllEdgeSet):
                if NodesTo[m] == NodeSetF[l]:
                    v = NodesFrom[m]
                    cons3B.add(x[v,NodesTo[m],k])
            model.addConstr(cons3B, gb.EQUAL, 0, "cons3B"+str(k)+"_"+str(l))

# Constraint 4 : Memastikan truk melakukan loading dan unloading
for k in range(nVehicles):
    for d in range(nNodeSetCD):
        v = NodeSetCD[d]
        cons4 = gb.LinExpr()
        for i in range(nAllEdgeSet):
            if NodesTo[i] == v:
                u = NodesFrom[i]
                cons4.add(x[u,v,k])
            if NodesFrom[i] == v:
                u = NodesTo[i]
                cons4.add(-1*x[v,u,k])
        model.addConstr(cons4, gb.EQUAL, 0, "cons4")

           # Constraint 5: Memastikan truk hanya unload di satu customer
for k in range(nVehicles):
    for c in range(nNodeSetC):
        u = NodeSetC[c]
        cons5 = grb.LinExpr()

        for i in range(nAllEdgeSet):
            if NodesFrom[i] == u:
                v = NodesTo[i]
                cons5.add(x[u, v, k])

        model.addConstr(cons5, grb.GRB.LESS_EQUAL, 1, f"cons5_{k}_{c}")

            # Constraint 6: Ensure delivery sequence for each customer
for i in range(nCustomerNode):
    for j in range(MaxNumDeliveryC[i] - 1):
        cons6 = grb.LinExpr()
        c_i_j_plus_1 = 0
        c_i_j = 0
        for k in range(nAllNodeSetIndex):
            if AllNodeSet[k] == CustomerNode[i] and AllNodeSetSequence[k] == j:
                c_i_j = AllNodeSetIndex[k]
            if AllNodeSet[k] == CustomerNode[i] and AllNodeSetSequence[k] == j + 1:
                c_i_j_plus_1 = AllNodeSetIndex[k]

        for k in range(nVehicles):
            for l in range(nAllEdgeSet):
                if NodesFrom[l] == CustomerNode[i] and NodesTo[l] == c_i_j_plus_1:
                    cons6.addTerms(1, x[c_i_j_plus_1][NodesTo[l]][k])
                if NodesFrom[l] == CustomerNode[i] and NodesTo[l] == c_i_j:
                    cons6.addTerms(-1, x[c_i_j][NodesTo[l]][k])

        model.addConstr(cons6, GRB.LESS_EQUAL, 0, f"cons6{i}{j}")

            # Constraint 7: Ensure each demand is satisfied
for i in range(nCustomerNode):
    cons7 = grb.LinExpr()
    u = CustomerNode[i]
    for k in range(nVehicles):
        for l in range(nAllEdgeSet):
            if NodesFrom[l] == u:
                v = NodesTo[l]
                cons7.add(VehicleCapacity[k] * x[u, v, k])
    cons7.add(-Demand[i] * y[i])
    model.addConstr(cons7, grb.GRB.GREATER_EQUAL, 0, f"cons7_{i}")
    # model.addConstr(cons7, grb.GRB.GREATER_EQUAL, Demand[i], f"cons7_{i}")

# Constraint 8: Ensure trucks only load at one depot
for i in range(nNodeSetD):
    u = NodeSetD[i]
    cons8 = grb.LinExpr()
    for k in range(nVehicles):
        for l in range(nAllEdgeSet):
            if NodesFrom[l] == u:
                v = NodesTo[l]
                cons8.add(x[u, v, k])
    model.addConstr(cons8, grb.GRB.LESS_EQUAL, 1, f"cons8_{i}")

# Constraint 9: Connect variables x and w using travel time and service time using a big constant M
# define big constant M
M = 1000
for i in range(nAllEdgeSet):
    u = NodesFrom[i]
    v = NodesTo[i]
    for k in range(nVehicles):
        # define periods here
        cons9 = grb.LinExpr()
        cons9.addConstant(-M)
        cons9.add(M * x[u, v, k])
        cons9.addConstant(AllNodeServiceTime[u]) # service time all depot and customers are the same
        cons9.addConstant(TravelTimeEdge[i])
        cons9.add(w[u])
        cons9.add(-w[v])
        model.addConstr(cons9, grb.GRB.LESS_EQUAL, 0, "cons9")

# Constraint 10: Ensure travel time is not greater than maximum concrete durability time
for i in range(nNodeSetD):
    for j in range(nNodeSetC):
        u = NodeSetD[i]
        v = NodeSetC[j]
        for k in range(nVehicles):
            cons10 = grb.LinExpr()
            cons10.addConstant(M)
            cons10.add(-M * x[u, v, k])
            cons10.addConstant(MaxTimeConcrete)
            cons10.addConstant(AllNodeServiceTime[u]) # service time all depot and customers are the same
            cons10.add(w[u])
            cons10.add(-w[v])
            model.addConstr(cons10, grb.GRB.GREATER_EQUAL, 0, "cons10")


            # Constraint 11: Ensure delivery schedule meets early time windows
for u in range(nNodeSetC):
    cons11 = gp.LinExpr()
    cons11.addTerms(1.0, w[NodeSetC[u]])
    model.addConstr(cons11, GRB.GREATER_EQUAL, timeStartC[u], f"cons11{u}")

for u in range(nNodeSetD):
    cons11 = gp.LinExpr()
    cons11.addTerms(1.0, w[NodeSetD[u]])
    model.addConstr(cons11, GRB.GREATER_EQUAL, timeStartD[u], f"cons11{u}")


# Constraint 12: Ensure delivery schedule meets late time windows
for u in range(nNodeSetC):
    cons12 = gp.LinExpr()
    cons12.addTerms(1.0, w[NodeSetC[u]])
    model.addConstr(cons12, GRB.LESS_EQUAL, timeEndC[u], f"cons12{u}")

for u in range(nNodeSetD):
    cons12 = gp.LinExpr()
    cons12.addTerms(1.0, w[NodeSetD[u]])
    model.addConstr(cons12, GRB.LESS_EQUAL, timeEndD[u], f"cons12{u}")


# Constraint 13: Ensure first delivery deadline
for u in range(nNodeSetC):
    for j in range(nAllNodeSetIndex):
        if AllNodeSetIndex[j] == NodeSetC[u] and AllNodeSetSequence[j] == 0:
            cons13 = gp.LinExpr()
            cons13.addTerms(1.0, w[NodeSetC[u]])
            model.addConstr(cons13, GRB.LESS_EQUAL, FirstDelivery[u], f"cons13{u}")


# Constraint 14: Ensure minimum time between deliveries for customers
for i in range(nCustomerNode):
    for j in range(MaxNumDeliveryC[i]-1):
        cons14 = gp.LinExpr()
        for k in range(nAllNodeSetIndex):
            if AllNodeSet[k] == CustomerNode[i] and AllNodeSetSequence[k] == j:
                cons14.addTerms(-1.0, w[k])
            if AllNodeSet[k] == CustomerNode[i] and AllNodeSetSequence[k] == j+1:
                cons14.addTerms(1.0, w[k])
        model.addConstr(cons14, GRB.GREATER_EQUAL, mintlC[i], f"cons14{i}")


# Constraint 15: Ensure minimum time between deliveries for depots
for i in range(nDepotNode):
    for j in range(MaxNumDeliveryD[i]-1):
        cons15 = gp.LinExpr()
        for k in range(nAllNodeSetIndex):
            if AllNodeSet[k] == DepotNode[i] and AllNodeSetSequence[k] == j:
                cons15.addTerms(-1.0, w[k])
            if AllNodeSet[k] == DepotNode[i] and AllNodeSetSequence[k] == j+1:
                cons15.addTerms(1.0, w[k])
        model.addConstr(cons15, GRB.GREATER_EQUAL, mintlD[i], f"cons15{i}")


# Constraint 16: Ensure maximum time between deliveries for customers
for i in range(nCustomerNode):
    for j in range(MaxNumDeliveryC[i]-1):
        cons16 = gp.LinExpr()
        for k in range(nAllNodeSetIndex):
            if All


IndentationError: ignored

In [ ]:
status = model.status
if status == GRB.OPTIMAL:
    print('Optimal objective value:', model.objVal)
    for v in model.getVars():
        print(v.varName, v.x)
elif status == GRB.INF_OR_UNBD:
    print('Model is infeasible or unbounded')
elif status == GRB.INFEASIBLE:
    print('Model is infeasible')
elif status == GRB.UNBOUNDED:
    print('Model is unbounded')
else:
    print('Optimization ended with status', status)

In [ ]:

            # Print solution
print("\nTotal Cost:", model.getAttr(GRB.Attr.ObjVal))

print("\nTraveled Edges: ")
for k in range(nVehicles):
    print("\nVehicle", k, ":")
    print(" Origin:", NodeSetO[k], "[", AllNodeSet[NodeSetO[k]], ",", AllNodeSetSequence[NodeSetO[k]], "]")
    print(" Finish:", NodeSetF[k], "[", AllNodeSet[NodeSetF[k]], ",", AllNodeSetSequence[NodeSetF[k]], "]")
    for i in range(nAllEdgeSet):
        u = NodesFrom[i]
        v = NodesTo[i]
        if x[u, v, k].x > 0.0000:
            print("x [", u, "][", v, "][", k, "] = 1")
            print("  x [", AllNodeSet[u], ",", AllNodeSetSequence[u], "][", AllNodeSet[v], ",", AllNodeSetSequence[v], "][", k, "] = 1")
        else:
            pass

print("\nFulfilled Demand: ")
for q in range(nDemand):
    if y[q].x > 0.0000:
        print("y[", q, "] = 1")
    else:
        print("y[", q, "] = 0")

print("\nSchedule: ")
for u in range(nAllNodeSetIndex):
    if w[u].x > 0.0000:
        print("w[", u, "] = ", w[u].x)

# Dispose of model and environment
model.dispose()
env.dispose()
